#  **C3 Diel leaf model**

This file discussesmaking C3 model from PlantCoreMetabolism model by applying required constraints

#Step:1  Install cobrapy
* Run %pip install cobra to install cobrapy

In [ ]:
%pip install cobra --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 46.4 MB/s eta 0:00:00


#Step:2 Import C3_model from Github
* Use wget using the syntax !wget link-to-file to retrieve the models and functions required for this analysis



In [ ]:
# Retrieving the model
!wget -q https://raw.githubusercontent.com/sshameer/DielLeafModeling/refs/heads/main/C3_model.sbml


# Step: 3 Import functions required to read sbml and perform pFBA
* A cobra model can be generated from an SBML file using the` read_sbml_model` function in the cobrapy io module
* pFBA can be performed using the pfba package in the cobrapy `flux_analysis` module

In [ ]:
from cobra.io import read_sbml_model
from cobra.flux_analysis import pfba

#Step:4 Import the model and constraining the model for diel leaf model of C3 photosynthesis
* The syntax for loading the model is `model_name = read_sbml_model(file_name)`

In [ ]:
C3_model = read_sbml_model("C3_model.sbml")

In [ ]:
# constraining GPT transport reaction
C3_model.reactions.get_by_id("G6P_Pi_pc1").lower_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc1").upper_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc2").lower_bound= 0
C3_model.reactions.get_by_id("G6P_Pi_pc2").upper_bound= 0

In [ ]:
# constraining sucrose accumulation to model a starch storing leaf
C3_model.reactions.get_by_id("SUCROSE_v_dielTransfer").bounds = (0,0)

In [ ]:
# constraining starch phosphorylation
C3_model.reactions.get_by_id("RXN_1826_p1").lower_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p1").upper_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p2").lower_bound= 0
C3_model.reactions.get_by_id("RXN_1826_p2").upper_bound= 0

* Maintanence cost defined by the refernce of [Töpfer et al.,2020](https://doi.org/10.1105/tpc.20.00132)
* The syntax is `solution_name = pfba(model_name)`
* The solution object has an attribute fluxes which containts a dictionary of all fluxes

In [ ]:
# Setting photon uptake
PPFD = 500
C3_model.reactions.Photon_tx1.upper_bound = PPFD
C3_model.reactions.Photon_tx1.lower_bound = 0
# Setting maintenance cost
VATPase = 0.0049*PPFD+2.7851 #Töpfer et al.,2020
C3_model.reactions.get_by_id("ATPase_tx1").bounds = (VATPase,VATPase)
C3sol = pfba(C3_model)

# Step:5 get the summary of the model after optimization
Run the syntax model_name.summary() for getting the summary of the model

In [ ]:
C3_model.summary()
# C3_model

Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e1,CO2_tx1,35.14,1,100.00%
WATER_e1,H2O_tx1,31.55,0,0.00%
NITRATE_e1,Nitrate_tx1,0.2171,0,0.00%
NITRATE_e2,Nitrate_tx2,0.1447,0,0.00%
OXYGEN_MOLECULE_e2,O2_tx2,2.023,0,0.00%
Photon_e1,Photon_tx1,500,0,0.00%
SULFATE_e1,SO4_tx1,0.002766,0,0.00%
PROTON_c1,unlProtHYPO_c1,0.3151,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
CARBON_DIOXIDE_e2,CO2_tx2,-1.846,1,100.00%


From uptake fluxes we can see that the model reports an uptake of CO2, and photon during the day and O2 at night. The model also reports release of O2 during the day and CO2 at night, indicative of C3

## checking reaction fluxes



In [ ]:
print(C3_model.reactions.get_by_id("diel_biomass").reaction)
print(C3_model.reactions.get_by_id("diel_biomass").flux)

3.0 X_Phloem_contribution_t1 + X_Phloem_contribution_t2 --> 
0.8258313952247737
